# RF Complexity Analysis (EU)
## Data Loading

We use the `forest_report.json` file in the repo. Each entry contains the dataset name, metadata (sizes, series length, split), and trained forest statistics.

Il conteggio delle ragioni (`R`, `NR`, ecc.) viene estratto una sola volta: il risultato viene salvato in `results/redis_reason_counts.csv` e riutilizzato ai run successivi. Imposta `FORCE_RESULTS_REFRESH=1` (o passa `refresh=True` al loader) per rigenerare il file quando arrivano nuovi risultati.

In [1]:
from __future__ import annotations

import json
import os
from collections import defaultdict
import base64
import binascii
import zipfile
from datetime import datetime
from pathlib import Path
from typing import Any, Mapping

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8-whitegrid')

# Optional styling (if unavailable, neutral fallback)
try:
    from etl.table_styling import style_summary_table, print_color_legend
except Exception:
    def style_summary_table(df: pd.DataFrame):
        return df.style
    def print_color_legend():
        pass

try:
    from etl.drifts_results import (
        compute_counts_from_results,
        load_analyzed_df,
        cast_dataset_str,
        DISPLAY_CATEGORIES,
        DISPLAY_NAMES,
        DISPLAY_LABELS,
    )
except Exception as exc:
    compute_counts_from_results = None
    load_analyzed_df = None
    cast_dataset_str = None
    DISPLAY_CATEGORIES = []
    DISPLAY_NAMES = {}
    DISPLAY_LABELS = {}
    print(f"Impossibile importare etl.drifts_results: {exc}")

CACHE_REFRESH_ENV = 'FORCE_RESULTS_REFRESH'
SUMMARY_FILENAME = 'redis_reason_counts.csv'
COUNTS_CACHE_FILENAME = '_counts_cache.csv'
META_FILENAME = 'redis_counts_meta.json'

REFRESH_TRUE = {'1', 'true', 'yes', 'y', 'on'}

def detect_base_dir() -> Path:
    try:
        from IPython import get_ipython
        ip = get_ipython()
        if ip is not None:
            pwd = ip.run_line_magic('pwd', '')
            if pwd:
                return Path(pwd).resolve()
    except Exception:
        pass
    return Path.cwd().resolve()

def _ensure_cache_dir(results_dir: Path) -> Path:
    base = results_dir / '_cache' if results_dir.exists() else (Path.cwd() / '_results_cache')
    base.mkdir(parents=True, exist_ok=True)
    return base

def _latest_results_mtime(results_dir: Path) -> float:
    if not results_dir.exists():
        return 0.0
    mtimes: list[float] = []
    for entry in results_dir.iterdir():
        try:
            mtimes.append(entry.stat().st_mtime)
        except OSError:
            continue
    return max(mtimes, default=0.0)

def _should_use_cache(summary_path: Path, counts_cache: Path, meta_path: Path, results_dir: Path, refresh_flag: bool) -> tuple[bool, dict[str, Any]]:
    if refresh_flag or not summary_path.exists() or not meta_path.exists() or not counts_cache.exists():
        return False, {}
    try:
        meta = json.loads(meta_path.read_text(encoding='utf-8'))
    except Exception:
        return False, {}
    latest_input = _latest_results_mtime(results_dir)
    cached_source = meta.get('source_mtime', 0.0)
    if latest_input and latest_input > cached_source:
        return False, {}
    return True, meta

def load_forest_report(path: Path) -> list[dict[str, Any]]:
    if not path.exists():
        raise FileNotFoundError(f'File not found: {path}')
    with path.open('r', encoding='utf-8') as handle:
        return json.load(handle)

def _load_cached_summary(summary_path: Path) -> pd.DataFrame:
    summary = pd.read_csv(summary_path)
    summary['dataset'] = summary['dataset'].astype(str)
    return summary

def _build_counts_from_summary(summary: pd.DataFrame) -> pd.DataFrame:
    if summary.empty:
        return pd.DataFrame()
    label_map = {cat: DISPLAY_LABELS.get(cat, DISPLAY_NAMES.get(cat, cat)) for cat in DISPLAY_CATEGORIES}
    label_to_cat = {label: cat for cat, label in label_map.items()}
    cols = [col for col in summary.columns if col in label_to_cat]
    if not cols:
        return pd.DataFrame()
    counts = summary[['dataset', *cols]].rename(columns=label_to_cat)
    return counts


class RedisDumpDecodeError(RuntimeError):
    """Raised when a Redis DUMP payload cannot be decoded."""

_RDB_ENCODING_INT8 = 0
_RDB_ENCODING_INT16 = 1
_RDB_ENCODING_INT32 = 2
_RDB_ENCODING_LZF = 3

def _split_dump_sections(raw: bytes) -> tuple[bytes, int, bytes]:
    if len(raw) < 10:
        raise RedisDumpDecodeError('DUMP payload is too short')
    checksum = raw[-8:]
    version = int.from_bytes(raw[-10:-8], 'little', signed=False)
    payload = raw[:-10]
    return payload, version, checksum

def _read_length_info(buffer: bytes, offset: int) -> tuple[int | None, int | None, int]:
    if offset >= len(buffer):
        raise RedisDumpDecodeError('Offset out of range while reading length')
    first = buffer[offset]
    prefix = first >> 6
    if prefix == 0:
        return first & 0x3F, None, offset + 1
    if prefix == 1:
        if offset + 1 >= len(buffer):
            raise RedisDumpDecodeError('Truncated 14-bit encoded length')
        second = buffer[offset + 1]
        length = ((first & 0x3F) << 8) | second
        return length, None, offset + 2
    if prefix == 2:
        if offset + 4 >= len(buffer):
            raise RedisDumpDecodeError('Truncated 32-bit encoded length')
        length = int.from_bytes(buffer[offset + 1 : offset + 5], 'big', signed=False)
        return length, None, offset + 5
    return None, first & 0x3F, offset + 1

def _lzf_decompress(data: bytes, expected_length: int) -> bytes:
    output = bytearray()
    idx = 0
    data_len = len(data)
    while idx < data_len:
        ctrl = data[idx]
        idx += 1
        if ctrl < 32:
            literal_len = ctrl + 1
            if idx + literal_len > data_len:
                raise RedisDumpDecodeError('Truncated literal LZF sequence')
            output.extend(data[idx : idx + literal_len])
            idx += literal_len
            continue
        length = ctrl >> 5
        ref_offset = len(output) - ((ctrl & 0x1F) << 8) - 1
        if length == 7:
            if idx >= data_len:
                raise RedisDumpDecodeError('Truncated LZF sequence while extending length')
            length += data[idx]
            idx += 1
        if idx >= data_len:
            raise RedisDumpDecodeError('Truncated LZF sequence while resolving reference')
        ref_offset -= data[idx]
        idx += 1
        length += 2
        if ref_offset < 0:
            raise RedisDumpDecodeError('Negative LZF reference')
        for _ in range(length):
            if ref_offset >= len(output):
                raise RedisDumpDecodeError('LZF reference out of range')
            output.append(output[ref_offset])
            ref_offset += 1
    if len(output) != expected_length:
        raise RedisDumpDecodeError('Unexpected decompressed length')
    return bytes(output)

def _decode_special_encoding(buffer: bytes, offset: int, encoding: int) -> tuple[bytes, int]:
    if encoding == _RDB_ENCODING_INT8:
        if offset >= len(buffer):
            raise RedisDumpDecodeError('Truncated 8-bit encoded integer')
        value = int.from_bytes(buffer[offset : offset + 1], 'little', signed=True)
        return str(value).encode('ascii'), offset + 1
    if encoding == _RDB_ENCODING_INT16:
        if offset + 2 > len(buffer):
            raise RedisDumpDecodeError('Truncated 16-bit encoded integer')
        value = int.from_bytes(buffer[offset : offset + 2], 'little', signed=True)
        return str(value).encode('ascii'), offset + 2
    if encoding == _RDB_ENCODING_INT32:
        if offset + 4 > len(buffer):
            raise RedisDumpDecodeError('Truncated 32-bit encoded integer')
        value = int.from_bytes(buffer[offset : offset + 4], 'little', signed=True)
        return str(value).encode('ascii'), offset + 4
    if encoding == _RDB_ENCODING_LZF:
        compressed_len, enc, next_offset = _read_length_info(buffer, offset)
        if enc is not None:
            raise RedisDumpDecodeError('Unexpected encoding for LZF length')
        data_len, enc, data_offset = _read_length_info(buffer, next_offset)
        if enc is not None:
            raise RedisDumpDecodeError('Unexpected encoding for LZF payload length')
        if compressed_len is None or data_len is None:
            raise RedisDumpDecodeError('Invalid LZF length encoding')
        end = data_offset + compressed_len
        if end > len(buffer):
            raise RedisDumpDecodeError('Truncated encoded string payload')
        compressed = buffer[data_offset:end]
        decompressed = _lzf_decompress(compressed, data_len)
        return decompressed, end
    raise RedisDumpDecodeError('Unknown string encoding')

def _read_encoded_string(buffer: bytes, offset: int) -> tuple[bytes, int]:
    length, encoding, next_offset = _read_length_info(buffer, offset)
    if encoding is None:
        if length is None:
            raise RedisDumpDecodeError('Missing length for raw string')
        end = next_offset + length
        if end > len(buffer):
            raise RedisDumpDecodeError('Truncated encoded string payload')
        return buffer[next_offset:end], end
    return _decode_special_encoding(buffer, next_offset, encoding)

def _decode_dump_string(entry: Mapping[str, Any]) -> bytes | None:
    value = entry.get('value')
    if not isinstance(value, Mapping):
        return None
    data_b64 = value.get('data')
    if not isinstance(data_b64, str):
        return None
    try:
        raw = base64.b64decode(data_b64.encode('ascii'))
    except (binascii.Error, UnicodeEncodeError):
        return None
    try:
        payload, _, _ = _split_dump_sections(raw)
        if not payload or payload[0] != 0:
            return None
        decoded, _ = _read_encoded_string(payload, 1)
        return decoded
    except RedisDumpDecodeError:
        return None

def _decode_worker_id(entry: Mapping[str, Any]) -> str | None:
    key_b64 = entry.get('key')
    if not isinstance(key_b64, str):
        return None
    try:
        raw = base64.b64decode(key_b64.encode('ascii'))
    except (binascii.Error, UnicodeEncodeError):
        return None
    text = raw.decode('utf-8', errors='replace')
    if not text:
        return None
    if ':' in text:
        return text.rsplit(':', 1)[0]
    return text

def _load_db10_entries_from_zip(zip_path: Path) -> list[Mapping[str, Any]]:
    try:
        with zipfile.ZipFile(zip_path, 'r') as archive:
            names = [name for name in archive.namelist() if 'redis_backup_db10' in name]
            entries: list[Mapping[str, Any]] = []
            for name in names:
                try:
                    payload = json.loads(archive.read(name).decode('utf-8'))
                except Exception:
                    continue
                entries.extend(payload.get('entries') or [])
            return entries
    except (FileNotFoundError, zipfile.BadZipFile):
        return []

def _load_db10_entries_from_dir(directory: Path) -> list[Mapping[str, Any]]:
    entries: list[Mapping[str, Any]] = []
    for path in directory.rglob('redis_backup_db10.json'):
        try:
            payload = json.loads(path.read_text(encoding='utf-8'))
        except Exception:
            continue
        entries.extend(payload.get('entries') or [])
    return entries

def _aggregate_worker_can_times(entries: list[Mapping[str, Any]]) -> dict[str, float]:
    totals: dict[str, float] = {}
    for entry in entries:
        decoded = _decode_dump_string(entry)
        if decoded is None:
            continue
        try:
            payload = json.loads(decoded.decode('utf-8', errors='replace'))
        except json.JSONDecodeError:
            continue
        worker_id = payload.get('worker_id') or _decode_worker_id(entry)
        if not worker_id:
            continue
        can_processing = payload.get('can_processing') or {}
        if not isinstance(can_processing, Mapping):
            continue
        time_value = can_processing.get('time')
        if time_value is None:
            time_value = can_processing.get('time_seconds')
        if time_value is None:
            continue
        try:
            seconds = float(time_value)
        except (TypeError, ValueError):
            continue
        totals[worker_id] = totals.get(worker_id, 0.0) + seconds
    return totals

def _safe_float(value: Any | None) -> float | None:
    try:
        return float(value)
    except (TypeError, ValueError):
        return None

def _safe_number(value: Any | None) -> float:
    result = _safe_float(value)
    return result if result is not None else 0.0


def _init_worker_metrics() -> dict[str, float]:
    return {
        'time_seconds': 0.0,
        'can_checks': 0.0,
        'iterations_total': 0.0,
        'iterations_good': 0.0,
        'iterations_bad': 0.0,
        'early_stop_good_total': 0.0,
        'early_stop_good_result_good': 0.0,
        'early_stop_good_result_bad': 0.0,
        'extensions_total_good': 0.0,
        'extensions_filtered_good': 0.0,
    }

def _iter_db10_entries(entry: Path) -> list[Mapping[str, Any]]:
    if entry.is_file() and entry.suffix.lower() == '.zip':
        try:
            with zipfile.ZipFile(entry, 'r') as archive:
                items: list[Mapping[str, Any]] = []
                for name in archive.namelist():
                    if 'redis_backup_db10' not in name:
                        continue
                    try:
                        payload = json.loads(archive.read(name).decode('utf-8'))
                    except Exception:
                        continue
                    items.extend(payload.get('entries') or [])
                return items
        except (FileNotFoundError, zipfile.BadZipFile):
            return []
    if entry.is_dir():
        items: list[Mapping[str, Any]] = []
        for path in entry.rglob('redis_backup_db10.json'):
            try:
                payload = json.loads(path.read_text(encoding='utf-8'))
            except Exception:
                continue
            items.extend(payload.get('entries') or [])
        return items
    return []

def _decode_db10_entry(entry: Mapping[str, Any]) -> Mapping[str, Any] | None:
    decoded = _decode_dump_string(entry)
    if decoded is None:
        return None
    try:
        return json.loads(decoded.decode('utf-8', errors='replace'))
    except json.JSONDecodeError:
        return None

def compute_worker_can_metrics(results_dir: Path) -> dict[str, dict[str, Any]]:
    dataset_stats: dict[str, dict[str, Any]] = {}
    all_time_values: list[float] = []
    overall_totals = defaultdict(float)
    overall_counts = {
        'worker_count': 0,
    }
    if not results_dir.exists():
        return {}
    for entry in sorted(results_dir.iterdir()):
        if entry.name.startswith('_'):
            continue
        dataset = entry.stem.split('_')[0] if entry.is_file() else entry.name.split('_')[0]
        if not dataset:
            continue
        db10_entries = _iter_db10_entries(entry)
        if not db10_entries:
            continue
        worker_metrics: dict[str, dict[str, float]] = {}
        for raw_entry in db10_entries:
            payload = _decode_db10_entry(raw_entry)
            if not isinstance(payload, Mapping):
                continue
            worker_id = payload.get('worker_id') or _decode_worker_id(raw_entry)
            if not worker_id:
                continue
            can_processing = payload.get('can_processing') or {}
            if not isinstance(can_processing, Mapping):
                continue
            metrics = worker_metrics.setdefault(worker_id, _init_worker_metrics())
            metrics['can_checks'] += 1.0
            time_value = can_processing.get('time')
            if time_value is None:
                time_value = can_processing.get('time_seconds')
            time_seconds = _safe_float(time_value)
            if time_seconds is not None:
                metrics['time_seconds'] += time_seconds
            result = str(can_processing.get('result') or '').upper()
            raw_info = can_processing.get('raw_info') or {}
            if isinstance(raw_info, Mapping):
                iterations_value = _safe_float(raw_info.get('iterations'))
                if iterations_value is not None:
                    metrics['iterations_total'] += iterations_value
                    if result == 'GOOD':
                        metrics['iterations_good'] += iterations_value
                    elif result == 'BAD':
                        metrics['iterations_bad'] += iterations_value
                early_stop_good = _safe_float(raw_info.get('early_stop_good'))
                if early_stop_good is not None and early_stop_good:
                    metrics['early_stop_good_total'] += early_stop_good
                    if result == 'GOOD':
                        metrics['early_stop_good_result_good'] += early_stop_good
                    elif result == 'BAD':
                        metrics['early_stop_good_result_bad'] += early_stop_good
            extensions = can_processing.get('extensions') or {}
            if result == 'GOOD' and isinstance(extensions, Mapping):
                metrics['extensions_total_good'] += _safe_number(extensions.get('total'))
                metrics['extensions_filtered_good'] += _safe_number(extensions.get('filtered'))
        if not worker_metrics:
            continue
        worker_count = len(worker_metrics)
        time_values = [m['time_seconds'] for m in worker_metrics.values() if m['time_seconds'] > 0.0]
        if time_values:
            all_time_values.extend(time_values)
        aggregate_totals = defaultdict(float)
        for metrics in worker_metrics.values():
            for key, value in metrics.items():
                aggregate_totals[key] += float(value)
        total_iterations = aggregate_totals['iterations_total']
        aggregate = {
            'worker_count': worker_count,
            'total_time_max': float(max(time_values)) if time_values else None,
            'total_time_mean': float(np.mean(time_values)) if time_values else None,
            'icf_checks': float(aggregate_totals['can_checks']),
            'reason_iterations_total': float(total_iterations),
            'reason_iterations_good': float(aggregate_totals['iterations_good']),
            'reason_iterations_bad': float(aggregate_totals['iterations_bad']),
            'iter_good_ratio': float(aggregate_totals['iterations_good'] / total_iterations) if total_iterations else None,
            'iter_bad_ratio': float(aggregate_totals['iterations_bad'] / total_iterations) if total_iterations else None,
            'earlystop_good_total': float(aggregate_totals['early_stop_good_total']),
            'esg': float(aggregate_totals['early_stop_good_result_good']),
            'esb': float(aggregate_totals['early_stop_good_result_bad']),
            'filtrered_total': float(aggregate_totals['extensions_total_good']),
            'filtrered_filtered': float(aggregate_totals['extensions_filtered_good']),
            'filtrered_rate': float(aggregate_totals['extensions_filtered_good'] / aggregate_totals['extensions_total_good']) if aggregate_totals['extensions_total_good'] else None,
        }
        dataset_stats[dataset] = {
            'workers': worker_metrics,
            'aggregate': aggregate,
        }
        overall_counts['worker_count'] += worker_count
        for key in (
            'can_checks',
            'iterations_total',
            'iterations_good',
            'iterations_bad',
            'early_stop_good_total',
            'early_stop_good_result_good',
            'early_stop_good_result_bad',
            'extensions_total_good',
            'extensions_filtered_good',
        ):
            overall_totals[key] += aggregate_totals[key]
    if dataset_stats:
        total_iterations = overall_totals['iterations_total']
        overall_aggregate = {
            'worker_count': overall_counts['worker_count'],
            'total_time_max': float(max(all_time_values)) if all_time_values else None,
            'total_time_mean': float(np.mean(all_time_values)) if all_time_values else None,
            'icf_checks': float(overall_totals['can_checks']),
            'reason_iterations_total': float(total_iterations),
            'reason_iterations_good': float(overall_totals['iterations_good']),
            'reason_iterations_bad': float(overall_totals['iterations_bad']),
            'iter_good_ratio': float(overall_totals['iterations_good'] / total_iterations) if total_iterations else None,
            'iter_bad_ratio': float(overall_totals['iterations_bad'] / total_iterations) if total_iterations else None,
            'earlystop_good_total': float(overall_totals['early_stop_good_total']),
            'esg': float(overall_totals['early_stop_good_result_good']),
            'esb': float(overall_totals['early_stop_good_result_bad']),
            'filtrered_total': float(overall_totals['extensions_total_good']),
            'filtrered_filtered': float(overall_totals['extensions_filtered_good']),
            'filtrered_rate': float(overall_totals['extensions_filtered_good'] / overall_totals['extensions_total_good']) if overall_totals['extensions_total_good'] else None,
        }
        dataset_stats['__overall__'] = {
            'workers': {},
            'aggregate': overall_aggregate,
        }
    return dataset_stats
def load_results_artifacts(
    results_dir: Path,
    forest_csv: Path,
    *,
    verbose: bool = True,
    refresh: bool | None = None,
    cache_dir: Path | None = None,
) -> dict[str, Any]:
    if cache_dir is None:
        cache_dir = _ensure_cache_dir(results_dir)
    else:
        cache_dir.mkdir(parents=True, exist_ok=True)

    summary_path = results_dir / SUMMARY_FILENAME
    counts_cache = cache_dir / COUNTS_CACHE_FILENAME
    meta_path = cache_dir / META_FILENAME

    env_flag = os.environ.get(CACHE_REFRESH_ENV, '').strip().lower()
    refresh_flag = refresh if refresh is not None else env_flag in REFRESH_TRUE

    use_cache, meta = _should_use_cache(summary_path, counts_cache, meta_path, results_dir, refresh_flag)

    if use_cache:
        if verbose:
            cached_at = meta.get('cached_at')
            print(f"Using cached redis summary (cached_at={cached_at})")
        summary = _load_cached_summary(summary_path)
        counts_df = _build_counts_from_summary(summary)
        if not counts_df.empty and isinstance(meta.get('log_summary'), dict):
            counts_df.attrs['log_summary'] = meta['log_summary']
        analyzed_df = load_analyzed_df(forest_csv) if load_analyzed_df else pd.DataFrame()
        return {
            'counts_df': counts_df,
            'analyzed_df': analyzed_df,
            'summary': summary,
            'results_datasets': set(meta.get('results_datasets', [])),
            'missing_zip_manifests': meta.get('missing_zip_manifests', []),
            'zip_dataset_prefixes': set(meta.get('zip_dataset_prefixes', [])),
            'log_summary': meta.get('log_summary', {}),
            'summary_path': summary_path,
            'used_cache': True,
        }

    if not compute_counts_from_results or not load_analyzed_df or not cast_dataset_str:
        empty = pd.DataFrame()
        return {
            'counts_df': empty,
            'analyzed_df': empty,
            'summary': empty,
            'results_datasets': [],
            'missing_zip_manifests': [],
            'zip_dataset_prefixes': [],
            'log_summary': {},
            'summary_path': summary_path,
            'used_cache': False,
        }

    counts_df = compute_counts_from_results(results_dir, verbose=verbose)
    analyzed_df = load_analyzed_df(forest_csv)

    counts_df = cast_dataset_str(counts_df)
    analyzed_df = cast_dataset_str(analyzed_df)

    if counts_df.empty:
        results_datasets: list[str] = []
    else:
        results_datasets = sorted(counts_df['dataset'].astype(str).unique())

    if results_datasets:
        analyzed_df_res = analyzed_df[analyzed_df['dataset'].isin(results_datasets)].copy()
        merged_results_only = analyzed_df_res.merge(counts_df, on='dataset', how='inner')
    else:
        merged_results_only = pd.DataFrame(columns=['dataset', *DISPLAY_CATEGORIES])

    if not merged_results_only.empty:
        for cat in DISPLAY_CATEGORIES:
            if cat not in merged_results_only.columns:
                merged_results_only[cat] = 0
        merged_results_only[DISPLAY_CATEGORIES] = (
            merged_results_only[DISPLAY_CATEGORIES]
            .fillna(0)
            .astype(int)
        )
        merged_results_only['TOT'] = merged_results_only[DISPLAY_CATEGORIES].sum(axis=1)
        merged_results_only = merged_results_only.sort_values('R', ascending=False)
        summary_cols = ['dataset', *DISPLAY_CATEGORIES, 'TOT']
        summary_cols = [c for c in summary_cols if c in merged_results_only.columns]
        summary = merged_results_only[summary_cols].copy()
        rename_map = {c: DISPLAY_LABELS.get(c, DISPLAY_NAMES.get(c, c)) for c in DISPLAY_CATEGORIES}
        rename_map['TOT'] = DISPLAY_LABELS.get('TOT', 'Total')
        summary = summary.rename(columns=rename_map)
    else:
        summary = pd.DataFrame(columns=['dataset', *DISPLAY_CATEGORIES])

    log_summary = counts_df.attrs.get('log_summary', {}) if hasattr(counts_df, 'attrs') else {}
    if not summary.empty and log_summary:
        summary['Worker start (min)'] = summary['dataset'].map(lambda ds: log_summary.get(ds, {}).get('log_start_min'))
        summary['Worker end (max)'] = summary['dataset'].map(lambda ds: log_summary.get(ds, {}).get('log_end_max'))
        summary['Worker span (s)'] = summary['dataset'].map(lambda ds: log_summary.get(ds, {}).get('log_duration_seconds'))
        if 'Worker span (s)' in summary.columns:
            summary['Worker span (s)'] = pd.to_numeric(summary['Worker span (s)'], errors='coerce').round(3)

    import zipfile

    missing: list[tuple[str, object]] = []
    zip_dataset_prefixes: set[str] = set()
    if results_dir.exists():
        for z in sorted(results_dir.glob('*.zip')):
            zip_dataset_prefixes.add(z.name.split('_')[0])
            try:
                with zipfile.ZipFile(z, 'r') as archive:
                    names = archive.namelist()
                    has_manifest = any('redis_backup_db' in name for name in names)
                    if not has_manifest:
                        missing.append((z.name, names[:10]))
            except Exception as exc:
                missing.append((z.name, f'error: {exc}'))

    cache_dir.mkdir(parents=True, exist_ok=True)
    try:
        summary.to_csv(summary_path, index=False)
        counts_df.to_csv(counts_cache, index=False)
        meta_payload = {
            'cached_at': datetime.utcnow().isoformat(),
            'source_mtime': _latest_results_mtime(results_dir),
            'results_datasets': results_datasets,
            'zip_dataset_prefixes': sorted(zip_dataset_prefixes),
            'missing_zip_manifests': missing,
            'log_summary': log_summary,
        }
        meta_path.write_text(json.dumps(meta_payload, ensure_ascii=False, indent=2), encoding='utf-8')
        if verbose:
            print(f"Saved redis summary cache at {summary_path}")
    except Exception as exc:
        if verbose:
            print(f"Warning: unable to persist redis summary caches ({exc})")

    return {
        'counts_df': counts_df,
        'analyzed_df': analyzed_df,
        'summary': summary,
        'results_datasets': results_datasets,
        'missing_zip_manifests': missing,
        'zip_dataset_prefixes': zip_dataset_prefixes,
        'log_summary': log_summary,
        'summary_path': summary_path,
        'used_cache': False,
    }

BASE_DIR = detect_base_dir()
RESULTS_DIR = BASE_DIR / 'results'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
FOREST_JSON = BASE_DIR / 'forest_report.json'
FOREST_CSV = BASE_DIR / 'forest_report.csv'

report_data = load_forest_report(FOREST_JSON)
results_artifacts = load_results_artifacts(RESULTS_DIR, FOREST_CSV, verbose=True)

summary = results_artifacts['summary']
counts_df = results_artifacts['counts_df']
analyzed_df = results_artifacts['analyzed_df']
results_datasets = set(results_artifacts.get('results_datasets', []))
zip_dataset_prefixes = set(results_artifacts.get('zip_dataset_prefixes', []))
missing_zip_manifests = results_artifacts.get('missing_zip_manifests', [])
log_summary = results_artifacts.get('log_summary', {})
saved_summary_path = results_artifacts.get('summary_path')
used_cache = bool(results_artifacts.get('used_cache'))

print(f'Base dir           : {BASE_DIR}')
print(f'Loaded {len(report_data)} rows from {FOREST_JSON}')
print(f'Results directory  : {RESULTS_DIR} (exists={RESULTS_DIR.exists()})')
if saved_summary_path:
    print(f'Summary cache path : {saved_summary_path}')
print(f'Using cached counts: {used_cache}')

Using cached redis summary (cached_at=2025-10-27T13:15:20.261007)
Base dir           : C:\Users\danie\Projects\GitHub\drifts
Loaded 88 rows from C:\Users\danie\Projects\GitHub\drifts\forest_report.json
Results directory  : C:\Users\danie\Projects\GitHub\drifts\results (exists=True)
Summary cache path : C:\Users\danie\Projects\GitHub\drifts\results\redis_reason_counts.csv
Using cached counts: True


## Summary Table

We build a table with: `dataset`, `analyzed` (if results exist in `results/`), dataset sizes (`train_size`, `test_size`, `series_length`), model size (`n_estimators`), and EU stats (`mean eu features`, `eu std`).

Correct sorting (priority):
- `n_estimators` ascending
- `eu_complexity` ascending (fallback to `series_length` when missing)
- `series_length` ascending
- `dataset` alphabetical


In [2]:
# Build summary table with EU metrics and result counts

def to_int(value: Any | None) -> int | None:
    try:
        return int(value)
    except (TypeError, ValueError):
        return None

def to_float(value: Any | None) -> float | None:
    try:
        return float(value)
    except (TypeError, ValueError):
        return None

def extract_metadata(entry: Mapping[str, Any]) -> dict[str, Any]:
    metadata = entry.get('metadata') if isinstance(entry.get('metadata'), Mapping) else {}
    statistics = entry.get('forest_statistics') if isinstance(entry.get('forest_statistics'), Mapping) else {}
    dataset = str(entry.get('dataset', '') or '').strip() or '<unknown>'
    return {
        'dataset': dataset,
        'n_estimators': to_int(statistics.get('n_estimators')),
        'series_length': to_int(metadata.get('series_length')),
        'train_size': to_int(metadata.get('train_size')),
        'test_size': to_int(metadata.get('test_size')),
        'avg_depth': to_float(statistics.get('avg_depth')),
        'avg_leaves': to_float(statistics.get('avg_leaves')),
        'avg_nodes': to_float(statistics.get('avg_nodes')),
    }

def build_eu_metrics(entries: list[Mapping[str, Any]]) -> dict[str, dict[str, Any]]:
    eu: dict[str, dict[str, Any]] = {}
    for e in entries:
        dataset = str(e.get('dataset', '') or '').strip()
        if not dataset:
            continue
        n_features = e.get('n_features') if isinstance(e.get('n_features'), (int, float)) else None
        mean_eu = e.get('mean_eu') if isinstance(e.get('mean_eu'), (int, float)) else None
        eu_complexity = e.get('eu_complexity') if isinstance(e.get('eu_complexity'), (int, float)) else None
        # Try deriving from endpoint dictionaries (consider common key variants)
        eu_obj = e.get('endpoints_universe') or e.get('endpoints') or e.get('endpoints_universe_summary')
        lengths = None
        eu_min = None
        eu_max = None
        eu_std_dev = None
        if isinstance(eu_obj, Mapping):
            lens: list[int] = []
            for _, endpoints in eu_obj.items():
                if isinstance(endpoints, (list, tuple)):
                    lens.append(len(endpoints))
            if lens:
                lengths = lens
                comp_n = len(lens)
                comp_mean = float(np.mean(lens))
                comp_cplx = comp_mean * comp_n
                comp_min = float(np.min(lens))
                comp_max = float(np.max(lens))
                comp_std = float(np.std(lens)) if len(lens) > 1 else 0.0
                if n_features is None:
                    n_features = comp_n
                if mean_eu is None:
                    mean_eu = comp_mean
                if eu_complexity is None:
                    eu_complexity = comp_cplx
                eu_min = comp_min
                eu_max = comp_max
                eu_std_dev = comp_std
        if n_features is not None and mean_eu is not None:
            entry = {
                'n_features': int(n_features),
                'mean_eu': float(mean_eu),
                'eu_complexity': float(eu_complexity) if eu_complexity is not None else float(mean_eu) * int(n_features),
                'lengths': lengths,
            }
            if eu_min is not None:
                entry['eu_min'] = float(eu_min)
            if eu_max is not None:
                entry['eu_max'] = float(eu_max)
            if eu_std_dev is not None:
                entry['eu_std_dev'] = float(eu_std_dev)
            eu[dataset] = entry
    return eu

# Build summary dataframe + EU metrics
summary_df = pd.DataFrame([extract_metadata(e) for e in report_data])
eu_metrics = build_eu_metrics(report_data)

summary_df['n_features'] = summary_df['dataset'].apply(lambda d: eu_metrics.get(d, {}).get('n_features'))
summary_df['mean_eu'] = summary_df['dataset'].apply(lambda d: eu_metrics.get(d, {}).get('mean_eu'))
summary_df['eu_complexity'] = summary_df['dataset'].apply(lambda d: eu_metrics.get(d, {}).get('eu_complexity'))
summary_df['eu_min'] = summary_df['dataset'].apply(lambda d: eu_metrics.get(d, {}).get('eu_min'))
summary_df['eu_max'] = summary_df['dataset'].apply(lambda d: eu_metrics.get(d, {}).get('eu_max'))
summary_df['eu_std_dev'] = summary_df['dataset'].apply(lambda d: eu_metrics.get(d, {}).get('eu_std_dev'))

# Harmonize display names for presentation
summary_df = summary_df.rename(
    columns={
        'mean_eu': 'mean eu features',
        'eu_std_dev': 'eu std',
    }
)

# Use the artifacts computed in the setup cell to mark analyzed datasets
analyzed_sources: set[str] = set()
if 'results_datasets' in globals():
    analyzed_sources.update(results_datasets)
if 'zip_dataset_prefixes' in globals():
    analyzed_sources.update(zip_dataset_prefixes)

if not analyzed_sources and 'RESULTS_DIR' in globals() and RESULTS_DIR.exists():
    analyzed_sources.update(p.name.split('_')[0] for p in RESULTS_DIR.glob('*.zip'))

if analyzed_sources:
    summary_df['analyzed'] = summary_df['dataset'].apply(lambda d: 'YES' if d in analyzed_sources else 'NO')
else:
    summary_df['analyzed'] = 'N/A'

# Sorting: n_estimators → eu_complexity (fallback series_length) → series_length → dataset
INF = float('inf')
summary_df['_sort_n_estimators'] = summary_df['n_estimators'].fillna(INF)
summary_df['_sort_eu'] = summary_df.apply(
    lambda r: r['eu_complexity'] if pd.notna(r.get('eu_complexity')) else (r['series_length'] if pd.notna(r.get('series_length')) else INF),
    axis=1,
)
summary_df['_sort_series_length'] = summary_df['series_length'].fillna(INF)
summary_df = (
    summary_df
    .sort_values(['_sort_n_estimators', '_sort_eu', '_sort_series_length', 'dataset'], ascending=[True, True, True, True])
    .drop(columns=['_sort_n_estimators', '_sort_eu', '_sort_series_length'])
)
summary_df = summary_df.reset_index(drop=True)

eu_only_df = summary_df[summary_df['eu_complexity'].notna()].copy()

# Merge reason/non-reason counts when available

RESULT_COUNT_COLUMNS = [
    'Total time (s) max',
    'Total time (s) mean',
    'ICF checks',
    'Reason check iteration total',
    'IterGoodRatio',
    'IterBadRatio',
    'Earlystop Good total',
    'ESG',
    'ESB',
    'Filtrered rate',
]
WORKER_CAN_COLUMN_MAP = {
    'total_time_max': 'Total time (s) max',
    'total_time_mean': 'Total time (s) mean',
    'icf_checks': 'ICF checks',
    'reason_iterations_total': 'Reason check iteration total',
    'iter_good_ratio': 'IterGoodRatio',
    'iter_bad_ratio': 'IterBadRatio',
    'earlystop_good_total': 'Earlystop Good total',
    'esg': 'ESG',
    'esb': 'ESB',
    'filtrered_rate': 'Filtrered rate',
}
summary_counts = pd.DataFrame(columns=['dataset', *RESULT_COUNT_COLUMNS])

if 'summary' in globals() and isinstance(summary, pd.DataFrame) and not summary.empty:
    summary_counts = summary.reindex(columns=['dataset', *RESULT_COUNT_COLUMNS]).copy()
    for col in RESULT_COUNT_COLUMNS:
        summary_counts[col] = pd.to_numeric(summary_counts[col], errors='coerce')

worker_can_metrics: dict[str, dict[str, Any]] = {}
WORKER_CAN_AGGREGATE_ROW: dict[str, Any] | None = None
if 'RESULTS_DIR' in globals():
    worker_can_metrics = compute_worker_can_metrics(RESULTS_DIR)

if worker_can_metrics:
    per_dataset_rows: list[dict[str, Any]] = []
    overall_aggregate: dict[str, Any] | None = None
    for dataset_key, payload in worker_can_metrics.items():
        if dataset_key == '__overall__':
            aggregate = payload.get('aggregate') if isinstance(payload, Mapping) else {}
            if isinstance(aggregate, Mapping):
                overall_aggregate = {key: aggregate.get(key) for key in WORKER_CAN_COLUMN_MAP}
            continue
        aggregate = payload.get('aggregate') if isinstance(payload, Mapping) else {}
        if not isinstance(aggregate, Mapping):
            continue
        row = {'dataset': str(dataset_key)}
        for agg_key, column_name in WORKER_CAN_COLUMN_MAP.items():
            row[column_name] = aggregate.get(agg_key)
        per_dataset_rows.append(row)
if per_dataset_rows:
    metrics_df = pd.DataFrame(per_dataset_rows).set_index('dataset')
    summary_counts = summary_counts.set_index('dataset')
    summary_counts = summary_counts.combine_first(metrics_df)
    summary_counts.update(metrics_df)
    summary_counts = summary_counts.reset_index()
    numeric_cols = list(WORKER_CAN_COLUMN_MAP.values())
    for col in numeric_cols:
        if col in summary_counts.columns:
            summary_counts[col] = pd.to_numeric(summary_counts[col], errors='coerce')
    for col in ['Total time (s) max', 'Total time (s) mean']:
        if col in summary_counts.columns:
            summary_counts[col] = summary_counts[col].round(3)
    for col in ['IterGoodRatio', 'IterBadRatio', 'Filtrered rate']:
        if col in summary_counts.columns:
            summary_counts[col] = summary_counts[col].round(6)
    if overall_aggregate:
        rounded_overall = overall_aggregate.copy()
        for key in ('total_time_max', 'total_time_mean'):
            if rounded_overall.get(key) is not None:
                rounded_overall[key] = round(float(rounded_overall[key]), 3)
        for key in ('iter_good_ratio', 'iter_bad_ratio', 'filtrered_rate'):
            if rounded_overall.get(key) is not None:
                rounded_overall[key] = round(float(rounded_overall[key]), 6)
        WORKER_CAN_AGGREGATE_ROW = rounded_overall
    else:
        WORKER_CAN_AGGREGATE_ROW = None
    WORKER_CAN_METRICS = worker_can_metrics
else:
    WORKER_CAN_AGGREGATE_ROW = None
counts_summary_df = summary_counts.copy()

# Column ordering for display
primary_columns = [
    'dataset',
    'analyzed',
    'train_size',
    'test_size',
    'series_length',
    'n_estimators',
    'mean eu features',
    'eu std',
]
available_primary = [col for col in primary_columns if col in summary_df.columns]
remaining_columns = [col for col in summary_df.columns if col not in available_primary]
summary_df = summary_df[available_primary + remaining_columns]

style_summary_table(summary_df)



,dataset,analyzed,train_size,test_size,series_length,n_estimators,mean eu features,eu std,avg_depth,avg_leaves,avg_nodes,n_features,eu_complexity,eu_min,eu_max
0,Wine,YES,57,54,234,10.000000,3.453488,0.709683,2.600000,4.400000,7.800000,86.000000,297.000000,3.000000,6.000000
1,Wafer,YES,1000,6164,152,10.000000,4.674419,1.629014,6.800000,13.200000,25.400000,129.000000,603.000000,3.000000,10.000000
2,MiddlePhalanxOutlineCorrect,YES,600,291,80,10.000000,18.000000,5.724945,8.000000,35.600000,70.200000,80.000000,1440.000000,7.000000,39.000000
3,MelbournePedestrian,YES,1138,2319,24,10.000000,60.666667,10.514540,9.300000,40.200000,79.400000,24.000000,1456.000000,37.000000,82.000000
4,ChlorineConcentration,NO,467,3840,166,10.000000,10.795181,3.718268,15.100000,39.400000,77.800000,166.000000,1792.000000,4.000000,25.000000
5,ScreenType,NO,375,375,720,10.000000,4.363333,1.420559,0.200000,1.200000,1.400000,600.000000,2618.000000,3.000000,10.000000
6,FordA,NO,3601,1320,500,10.000000,17.622000,4.114744,10.300000,30.000000,59.000000,500.000000,8811.000000,7.000000,31.000000
7,FordB,NO,3636,810,500,10.000000,17.928000,4.100587,12.200000,30.000000,59.000000,500.000000,8964.000000,8.000000,33.000000
8,ElectricDevices,NO,8926,7711,96,10.000000,310.906250,61.024257,26.600000,360.900000,720.800000,96.000000,29847.000000,189.000000,433.000000
9,SonyAIBORobotSurface1,YES,20,601,70,17.000000,3.312500,0.582961,1.647059,2.705882,4.411765,32.000000,106.000000,3.000000,5.000000


In [3]:
if 'summary_df' in globals() and isinstance(summary_df, pd.DataFrame) and 'dataset' in summary_df.columns:
    analyzed_datasets = summary_df.loc[summary_df.get('analyzed') == 'YES', 'dataset'].astype(str)
else:
    analyzed_datasets = pd.Series(dtype=str)
counts_display_cols = ['dataset', *RESULT_COUNT_COLUMNS]
if 'counts_summary_df' in globals():
    working_counts = counts_summary_df.copy()
else:
    working_counts = pd.DataFrame(columns=counts_display_cols)
if working_counts.empty:
    analyzed_counts_df = pd.DataFrame(columns=counts_display_cols)
else:
    working_counts['dataset'] = working_counts['dataset'].astype(str)
    analyzed_counts_df = working_counts[working_counts['dataset'].isin(analyzed_datasets)].copy()
if analyzed_counts_df.empty:
    analyzed_counts_df = pd.DataFrame(columns=counts_display_cols)

for col in counts_display_cols:
    if col not in analyzed_counts_df.columns:
        analyzed_counts_df[col] = pd.NA
analyzed_counts_df = analyzed_counts_df[counts_display_cols]

if (
    'WORKER_CAN_AGGREGATE_ROW' in globals()
    and WORKER_CAN_AGGREGATE_ROW
    and 'WORKER_CAN_COLUMN_MAP' in globals()
):
    overall_row = {'dataset': 'All workers'}
    for agg_key, column_name in WORKER_CAN_COLUMN_MAP.items():
        overall_row[column_name] = WORKER_CAN_AGGREGATE_ROW.get(agg_key)
    overall_df = pd.DataFrame([overall_row])
    for col in counts_display_cols:
        if col not in overall_df.columns:
            overall_df[col] = pd.NA
    overall_df = overall_df[counts_display_cols]
    for col in ['Total time (s) max', 'Total time (s) mean']:
        if col in overall_df.columns:
            overall_df[col] = pd.to_numeric(overall_df[col], errors='coerce').round(3)
    for col in ['IterGoodRatio', 'IterBadRatio', 'Filtrered rate']:
        if col in overall_df.columns:
            overall_df[col] = pd.to_numeric(overall_df[col], errors='coerce').round(6)
    analyzed_counts_df = pd.concat([analyzed_counts_df, overall_df], ignore_index=True, sort=False)

style_summary_table(analyzed_counts_df)


,dataset,Total time (s) max,Total time (s) mean,ICF checks,Reason check iteration total,IterGoodRatio,IterBadRatio,Earlystop Good total,ESG,ESB,Filtrered rate
0,ECG200,117779.088000,45937.485000,3507.000000,13825002.000000,0.870040,0.129960,6752764.000000,5877668.000000,875096.000000,0.284463
1,HandOutlines,31710.246000,5999.882000,5141.000000,2458537.000000,0.762682,0.237318,1071821.000000,803728.000000,268093.000000,0.911227
2,MelbournePedestrian,32398.297000,27536.924000,7789.000000,223435165.000000,1.000000,0.000000,104390152.000000,104390152.000000,0.000000,0.102971
3,MiddlePhalanxOutlineCorrect,16206.241000,1469.590000,976.000000,228277.000000,0.899407,0.100593,109833.000000,98584.000000,11249.000000,0.066880
4,SonyAIBORobotSurface1,153.292000,93.066000,32738.000000,37299.000000,0.715944,0.284056,7305.000000,5985.000000,1320.000000,0.940715
5,Wafer,54230.530000,35463.843000,6174.000000,196066056.000000,1.000000,0.000000,96575550.000000,96575550.000000,0.000000,0.141552
6,Wine,5375.737000,580.118000,2164.000000,235916.000000,0.205293,0.794707,116610.000000,24323.000000,92287.000000,0.416047
7,All workers,117779.088000,16725.844000,58489.000000,436286252.000000,0.994038,0.005962,209024035.000000,207775990.000000,1248045.000000,0.301091


In [4]:
# Display merged redis summary diagnostics using preloaded artifacts
print(f'? BASE_DIR     : {BASE_DIR}')
print(f'? RESULTS_DIR  : {RESULTS_DIR}')
print(f'? FOREST_REPORT: {FOREST_CSV.exists()}')
print(f'? SUMMARY_FILE : {saved_summary_path if saved_summary_path else "<none>"}')
print(f'? USED_CACHE   : {used_cache}')

if missing_zip_manifests:
    print('ZIP senza redis manifest (primi entry mostrati):')
    for name, sample in missing_zip_manifests:
        print('-', name, '->', sample)
elif RESULTS_DIR.exists():
    print('Tutti gli zip contengono redis_backup_db*.json (o non ci sono zip).')
else:
    print('Directory results non trovata.')

if results_datasets:
    print('Dataset conteggiati:', ', '.join(sorted(results_datasets)))
else:
    print(f"Nessun redis manifest valido trovato in {RESULTS_DIR}")

summary_to_show = summary.copy() if isinstance(summary, pd.DataFrame) and not summary.empty else pd.DataFrame(columns=['dataset', *DISPLAY_CATEGORIES])

try:
    display_obj = style_summary_table(summary_to_show)
    display_obj
except Exception:
    summary_to_show

print_color_legend()


? BASE_DIR     : C:\Users\danie\Projects\GitHub\drifts
? RESULTS_DIR  : C:\Users\danie\Projects\GitHub\drifts\results
? FOREST_REPORT: True
? SUMMARY_FILE : C:\Users\danie\Projects\GitHub\drifts\results\redis_reason_counts.csv
? USED_CACHE   : True
Tutti gli zip contengono redis_backup_db*.json (o non ci sono zip).
Dataset conteggiati: ECG200, HandOutlines, MelbournePedestrian, MiddlePhalanxOutlineCorrect, SonyAIBORobotSurface1, Wafer, Wine
COLUMN COLOR GRADIENTS
  n_estimators           -> Reds (min -> max)
  eu_complexity          -> Oranges (min -> max)
  series_length          -> YlOrBr (min -> max)
  n_features             -> YlGn (min -> max)
  mean_eu                -> Greens (min -> max)
  eu_min                 -> Greens (min -> max)
  eu_max                 -> Greens (min -> max)
  eu_std_dev             -> Greens (min -> max)
  Candidate              -> Purples (min -> max)
  Reason                 -> Greens (min -> max)
  Non-reason             -> Blues (min -> max)
  Candi

In [5]:
# Build combined table for analyzed datasets only (union of summary + CAN metrics)
combined_analyzed_df = pd.DataFrame()

if 'summary_df' in globals() and isinstance(summary_df, pd.DataFrame):
    if 'analyzed' in summary_df.columns:
        analyzed_summary = summary_df[summary_df['analyzed'] == 'YES'].copy()
    else:
        analyzed_summary = summary_df.copy()
else:
    analyzed_summary = pd.DataFrame()

if 'analyzed_counts_df' in globals() and isinstance(analyzed_counts_df, pd.DataFrame):
    counts_df = analyzed_counts_df.copy()
    if 'dataset' in counts_df.columns:
        counts_df['dataset'] = counts_df['dataset'].astype(str)
        counts_df = counts_df[counts_df['dataset'] != 'All workers']
    else:
        counts_df['dataset'] = pd.NA
else:
    counts_df = pd.DataFrame()

if not analyzed_summary.empty and 'dataset' in analyzed_summary.columns:
    analyzed_summary['dataset'] = analyzed_summary['dataset'].astype(str)
    combined_df = analyzed_summary.merge(counts_df, on='dataset', how='left', suffixes=('', '_worker_can'))
else:
    combined_df = counts_df.copy()

first_columns = [col for col in analyzed_summary.columns if col != 'analyzed'] if not analyzed_summary.empty else ['dataset']
if 'dataset' not in first_columns:
    first_columns = ['dataset', *first_columns]
second_columns = [col for col in counts_df.columns if col != 'dataset']
combined_columns = []
for col in first_columns:
    if col not in combined_columns:
        combined_columns.append(col)
for col in ['dataset', *second_columns]:
    if col not in combined_columns:
        combined_columns.append(col)

for col in combined_columns:
    if col not in combined_df.columns:
        combined_df[col] = pd.NA
combined_df = combined_df[combined_columns]

if 'analyzed' in combined_df.columns:
    combined_df = combined_df.drop(columns=['analyzed'])

if 'dataset' in combined_df.columns:
    combined_df = combined_df.set_index('dataset')

combined_analyzed_df = combined_df.copy()
combined_analyzed_df = combined_analyzed_df.transpose()
for col in combined_analyzed_df.columns:
    try:
        converted = pd.to_numeric(combined_analyzed_df[col])
    except (ValueError, TypeError):
        continue
    else:
        combined_analyzed_df[col] = converted

try:
    from etl import table_styling as _table_styling
    _color_map = getattr(_table_styling, 'COLUMN_COLORMAPS', {})
    _default_cmap = getattr(_table_styling, 'DEFAULT_CMAP', 'Greys')
except Exception:
    _color_map = {}
    _default_cmap = 'Greys'

styled = combined_analyzed_df.style
for metric in combined_analyzed_df.index:
    cmap = _color_map.get(metric, _default_cmap)
    styled = styled.background_gradient(subset=pd.IndexSlice[[metric], :], cmap=cmap, axis=1)

styled


dataset,Wine,Wafer,MiddlePhalanxOutlineCorrect,MelbournePedestrian,SonyAIBORobotSurface1,HandOutlines,ECG200
train_size,57.000000,1000.000000,600.000000,1138.000000,20.000000,1000.000000,100.000000
test_size,54.000000,6164.000000,291.000000,2319.000000,601.000000,370.000000,100.000000
series_length,234.000000,152.000000,80.000000,24.000000,70.000000,2709.000000,96.000000
n_estimators,10.000000,10.000000,10.000000,10.000000,17.000000,59.000000,101.000000
mean eu features,3.453488,4.674419,18.000000,60.666667,3.312500,3.252273,4.041667
eu std,0.709683,1.629014,5.724945,10.514540,0.582961,0.557995,1.147915
avg_depth,2.600000,6.800000,8.000000,9.300000,1.647059,9.338983,3.069307
avg_leaves,4.400000,13.200000,35.600000,40.200000,2.705882,20.000000,4.673267
avg_nodes,7.800000,25.400000,70.200000,79.400000,4.411765,39.000000,8.346535
n_features,86.000000,129.000000,80.000000,24.000000,32.000000,880.000000,72.000000
